# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = "../WeatherPy/cities.csv"

city_df = pd.read_csv(city_data)
city_df.dropna()
del city_df['Unnamed: 0']
city_df[['Latitude' , 'Longitude']] = city_df[['Latitude', 'Longitude']].apply(pd.to_numeric)
city_df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Laas,46.6166,10.7002,50.02,90,80,6.67,IT,2021-08-29 00:17:22
1,Butaritari,3.0707,172.7902,83.26,73,79,10.63,KI,2021-08-29 00:17:22
2,Punta Arenas,-53.1500,-70.9167,37.51,87,90,20.71,CL,2021-08-29 00:17:23
3,Alice Springs,-23.7000,133.8833,65.75,24,1,13.80,AU,2021-08-29 00:17:23
4,Qaanaaq,77.4840,-69.3632,36.32,93,100,4.59,GL,2021-08-29 00:17:24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
gmaps.configure(api_key=g_key)


In [32]:
locations_df = city_df[['Latitude', 'Longitude']].astype(float)
humidity_df = city_df['Humidity'].astype(float)

In [33]:
fig = gmaps.figure(center=[0,0], zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity_df, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
# creating my perfect weather conditions 
temp_file = city_df.loc[(city_df['Max Temp']>=60) & (city_df ["Max Temp"] <=85)]
wind_file = temp_file.loc[(temp_file['Wind Speed'] <=5)]
ideal_weather = wind_file.loc [(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,Mount Darwin,-16.7725,31.5838,61.66,45,0,3.33,ZW,2021-08-29 00:17:29
72,Dul'durga,50.6761,113.5931,60.33,55,0,3.31,RU,2021-08-29 00:17:42
88,Tiznit Province,29.5833,-9.5000,73.40,42,0,3.96,MA,2021-08-29 00:17:47
166,La Grande,45.3246,-118.0877,83.21,23,0,4.38,US,2021-08-29 00:16:09
177,Buşayrā,30.7376,35.6078,71.20,53,0,4.74,JO,2021-08-29 00:18:11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df['Hotel Name'] = ''

hotel_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Mount Darwin,-16.7725,31.5838,61.66,45,0,3.33,ZW,2021-08-29 00:17:29,
1,Dul'durga,50.6761,113.5931,60.33,55,0,3.31,RU,2021-08-29 00:17:42,
2,Tiznit Province,29.5833,-9.5000,73.40,42,0,3.96,MA,2021-08-29 00:17:47,
3,La Grande,45.3246,-118.0877,83.21,23,0,4.38,US,2021-08-29 00:16:09,
4,Buşayrā,30.7376,35.6078,71.20,53,0,4.74,JO,2021-08-29 00:18:11,


In [36]:
#search parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}



In [37]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: Mount Darwin.
Missing field/result... skipping.
------------
Retrieving Results for Index 1: Dul'durga.
Closest hotel in Dul'durga is Golden Dragon.
------------
Retrieving Results for Index 2: Tiznit Province.
Missing field/result... skipping.
------------
Retrieving Results for Index 3: La Grande.
Closest hotel in La Grande is Best Western Plus Rama Inn & Suites.
------------
Retrieving Results for Index 4: Buşayrā.
Closest hotel in Buşayrā is Ein Lahda Camp Site.
------------
Retrieving Results for Index 5: Leverano.
Closest hotel in Leverano is Palazzo Mastore.
------------
Retrieving Results for Index 6: Port Macquarie.
Closest hotel in Port Macquarie is Port Macquarie Hotel.
------------
Retrieving Results for Index 7: Marsá Maţrūḩ.
Closest hotel in Marsá Maţrūḩ is Royal Crown Hotel.
------------
Retrieving Results for Index 8: Manaus.
Closest hotel in Manaus is Quality Hotel Manaus.
------------
Retrieving Results for Index 9: Florence.
Closest ho

In [38]:
hotel_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Mount Darwin,-16.7725,31.5838,61.66,45,0,3.33,ZW,2021-08-29 00:17:29,
1,Dul'durga,50.6761,113.5931,60.33,55,0,3.31,RU,2021-08-29 00:17:42,Golden Dragon
2,Tiznit Province,29.5833,-9.5000,73.40,42,0,3.96,MA,2021-08-29 00:17:47,
3,La Grande,45.3246,-118.0877,83.21,23,0,4.38,US,2021-08-29 00:16:09,Best Western Plus Rama Inn & Suites
4,Buşayrā,30.7376,35.6078,71.20,53,0,4.74,JO,2021-08-29 00:18:11,Ein Lahda Camp Site


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [40]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))